In [ ]:
import numpy as np
import pandas as pd
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import yfinance as yf  # For real data reference (optional)
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
from transformers import pipeline

In [ ]:
# Task 1: Data Generation

# Generate Synthetic Stock Prices using GBM
def generate_stock_prices(num_days=252, num_stocks=5, mu=0.05, sigma=0.2, S0=100):
    dt = 1 / num_days
    stock_prices = np.zeros((num_days, num_stocks))
    stock_prices[0, :] = S0
    for t in range(1, num_days):
        stock_prices[t, :] = stock_prices[t - 1, :] * np.exp(
            (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * np.random.randn(num_stocks)
        )
    return pd.DataFrame(stock_prices, columns=[f'Stock_{i+1}' for i in range(num_stocks)])

In [ ]:
# Generate Mutual Fund NAVs
def generate_mutual_fund_navs(num_days=252, num_funds=3):
    navs = np.cumsum(np.random.randn(num_days, num_funds) * 0.5, axis=0) + 100
    return pd.DataFrame(navs, columns=[f'Fund_{i+1}' for i in range(num_funds)])

In [ ]:
# Generate User Risk Profiles
def generate_risk_profiles(num_users=100):
    risk_levels = ['Low', 'Medium', 'High']
    users = {
        'User_ID': range(1, num_users+1),
        'Risk_Profile': [random.choice(risk_levels) for _ in range(num_users)]
    }
    return pd.DataFrame(users)

In [ ]:
# Generate and save data
stock_data = generate_stock_prices()
mutual_fund_data = generate_mutual_fund_navs()
risk_profiles = generate_risk_profiles()

stock_data.to_csv('stock_data.csv', index=False)
mutual_fund_data.to_csv('mutual_fund_data.csv', index=False)
risk_profiles.to_csv('risk_profiles.csv', index=False)

In [ ]:
# Task 2: AI Agent for Investment Advisory

# NLP-Based Query Understanding
nlp_pipeline = pipeline("text-classification", model="ProsusAI/finbert")
def analyze_query(query):
    return nlp_pipeline(query)

In [ ]:
# Stock Price Forecasting with LSTM
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Portfolio Optimization using Markowitz MPT
def optimize_portfolio(stock_data):
    mu = mean_historical_return(stock_data)
    S = CovarianceShrinkage(stock_data).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    return cleaned_weights

In [ ]:
print("All tasks implemented successfully. Data saved as CSVs.")